In [ ]:
# %load load_data.py
from glob import glob
import numpy as np
import mne
from mne import create_info, concatenate_raws, Epochs, find_events
from mne.io import RawArray
from mne.channels import read_montage
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt


def describe_data(files):
    if len(files) == 0:
        print "no files found"
        return
    raw = get_raw(files)
    df = raw.to_data_frame()
    event_id = {'Non-Target': 1, 'Target': 2}
    reject = {'eeg': 100e-6}
    avg, epochs = clean_epochs(event_id, raw, 1, 30, reject)

    df.describe()
    avg.plot()
    avg.plot_image()

    for kind in event_id.keys():
        subset = epochs[kind]
        av = subset.average()
        ixs_plot = mne.pick_types(av.info, meg=False, eeg=True)
        _ = av.plot_joint(picks=ixs_plot, title=kind)


def get_raw(files):
    raw = []
    for file_name in files:
        # read the file
        data = pd.read_csv(file_name, index_col=0)
        sfreq = 256

        # name of each channels
        ch_names = list(data.columns)[0:4] + ['Stim']

        # type of each channels
        ch_types = ['eeg'] * 4 + ['stim']
        montage = read_montage('standard_1005')

        # get data and exclude Aux channel
        data = data.values[:, [0, 1, 2, 3, 5]].T

        # convert to Volts (from uVolts (microvolt) to volt
        # from the docs:
        # /muse/eeg is uV , floats, range: 0.0 - 1682.815 uV
        # 1uv == (10^-6) * (1 Volt)
        data[:-1] *= 1e-6

        # create mne objects
        info = create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq, montage=montage)
        raw.append(RawArray(data=data, info=info))

    # concatenates all raw objects
    raw = concatenate_raws(raw)
    return raw


def clean_epochs(event_id, raw, start, end, reject):
    # filter out events outside of start, end
    #  power line noise - 50 hz,
    raw.filter(start, end, method='iir')
    events = find_events(raw)

    epochs = Epochs(raw, events=events, event_id=event_id, tmin=-0.1, tmax=0.8, baseline=None,
                    reject=reject, preload=True, verbose=False, picks=[0, 1, 2, 3], add_eeg_ref=False)

    avg = epochs.average()
    avg.plot()

    return avg, epochs


def p300(epochs):
    epochs.pick_types(eeg=True)
    X = epochs.get_data() * 1e6
    times = epochs.times
    y = epochs.events[:, -1]

    sns.set_context('talk')
    sns.set_style('white')
    fig, axes = plt.subplots(2, 2, figsize=[12, 6], sharex=True, sharey=True)
    axes = [axes[1, 0], axes[0, 0], axes[0, 1], axes[1, 1]]

    for ch in range(4):
        sns.tsplot(X[y == 1, ch], time=times, n_boot=1000, ax=axes[ch])
        sns.tsplot(X[y == 2, ch], time=times, color='r', n_boot=1000, ax=axes[ch])
        axes[ch].set_title(epochs.ch_names[ch])
        axes[ch].set_ylim(-6, 6)

    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude (uV)')
    axes[-1].set_xlabel('Time (s)')
    axes[1].set_ylabel('Amplitude (uV)')
    plt.legend(['Non-target', 'Target'])
    sns.despine()
    plt.tight_layout()
    return plt


def p300_from_path(path, reject):
    files = glob(path)
    raw = get_raw(files)
    event_id = {'Non-Target': 1, 'Target': 2}
    avg, epochs = clean_epochs(event_id, raw, 1, 30, reject)
    return p300(epochs)


In [30]:
%%capture
# load the data into two raws and datasets ( capture - do not print to screen)
# files1 = glob('/Users/shiran/workspace/muse-lsl/data/visual/P300/subject%s/session%s/data_*.csv' % (1, 1))
files1 = glob('/Users/shiran/workspace/muse-lsl/data/visual/P300/subject%s/session%s/data_*.csv' % (1, 1))
files2 = glob("/Users/shiran/workspace/muse-lsl/data/visual/P300/shiran/dogs/data*.csv" )
#files2 = glob('/Users/shiran/workspace/muse-lsl/data/visual/P300/subject%s/session%s/data_2017-02-04-15_47_49.csv' % (1, 1))


raw1 = get_raw(files1)
raw2 = get_raw(files2)

df1 = raw1.to_data_frame()
df2 = raw2.to_data_frame()


event_id = {'Non-Target': 1, 'Target': 2}
reject = {'eeg': 100e-6}
avg1, epochs1 = clean_epochs(event_id, raw1, 1, 30, reject)
avg2, epochs2 = clean_epochs(event_id, raw2, 1, 30, reject)

In [31]:
from IPython.display import display
# display(df1.tail(10))
# df2.tail(10)

# display(df1[df1.Stim==1].describe())
# df2[df2.Stim==1].describe()
display(df1.describe())
display(df2.describe())

signal,TP9,AF7,AF8,TP10,Stim
count,184392.000000,184392.000000,184392.000000,184392.000000,184392.000000
mean,40.309915,28.733909,38.088774,62.160818,0.007300
std,53.261776,5.308971,17.634856,11.685295,0.096193
min,-309.082000,-39.062000,-1000.000000,-163.086000,0.000000
25%,-8.789000,25.391000,33.203000,54.688000,0.000000
50%,42.969000,28.809000,38.086000,62.012000,0.000000
75%,88.867000,31.738000,42.969000,69.824000,0.000000
max,254.883000,178.223000,999.512000,247.559000,2.000000


signal,TP9,AF7,AF8,TP10,Stim
count,61464.000000,61464.000000,61464.000000,61464.000000,61464.000000
mean,21.431053,30.157192,26.932527,20.151270,0.006768
std,61.200580,10.210899,94.562363,46.244194,0.089577
min,-601.074000,-10.254000,-216.309000,-1000.000000,0.000000
25%,-34.180000,22.461000,-31.250000,-17.090000,0.000000
50%,18.066000,29.785000,24.902000,18.555000,0.000000
75%,76.172000,37.598000,85.449000,57.129000,0.000000
max,296.387000,126.465000,298.828000,566.895000,2.000000


In [ ]:
from IPython.display import display
# path = "/Users/shiran/workspace/muse-lsl/data/visual/P300/shiran/*/data_2017-07-28-10.48.49.csv"
path = '/Users/shiran/workspace/muse-lsl/data/visual/P300/subject%s/session%s/data_*.csv' % (1, 1)
raw_data_uncliean=get_raw( glob(path))
#beofre cleaning
raw_data=get_raw( glob(path))
raw_data.filter(1,30)
display(raw_data.to_data_frame().describe())
display(raw_data_uncliean.to_data_frame().describe())

In [ ]:
display(raw1.plot_psd(tmax=np.inf))
raw2.plot_psd(tmax=np.inf)

In [ ]:
avg1.plot()
avg1.plot_image()
print()

In [ ]:
avg2.plot()
avg2.plot_image()
print()

In [ ]:
plt1 = p300(epochs1)
plt1.show()

plt2 = p300(epochs2)
plt2.show()

In [ ]:
# this block enales to hide the code block
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="toggle on/off the raw code."></form>''')